---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [20]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [21]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}


In [22]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    data = pd.read_csv('university_towns.txt',sep = '\n' , header = None)
    states = data[data[0].str.contains('edit')]
    states = states[0].str.replace(r"\[.*\]","").str.strip()
    data.iloc[240] = "Faribault, South Central College(Minnesota)"
    data.iloc[246] = "North Mankato, South Central College (Minnesota)"
    data.iloc[200] = "The Colleges of Worcester Consortium: (Massachusetts)"
    data.iloc[205] = "The Five College Region of Western Massachusetts: (Massachusetts)"
    
    data['RegionName'] = data[0].apply(lambda x: x.split(
    '(')[0].strip() if x.count('(') > 0 else np.NaN)
    
    data['State'] = data[0].apply(lambda x: x.split('[')[0].strip(
    ) if x.count('edit') > 0 else np.NaN).fillna(method="ffill")
    
    data = data.dropna().drop(0, axis = 1).reindex(
    columns=['State', 'RegionName']).reset_index(drop = True)
    return data


In [23]:
def import_data():
    gdp_data = pd.read_excel("gdplev.xls", header= None,  skiprows= 8 )
    
    gdp_data = gdp_data.loc[: , [4,6]]
    gdp_data.columns = ['Time', 'GDP']
    gdp_data = gdp_data.set_index('Time')
    gdp_data = gdp_data.loc['2000q1': ]
    gdp_data['GDP_Change'] = gdp_data.diff()
    return gdp_data

In [24]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp_data = import_data()
    gdp_data['RecessionStart'] = (gdp_data['GDP_Change'] < 0) & (gdp_data['GDP_Change'].shift(-1) < 0)

    gdp_data = gdp_data[gdp_data['RecessionStart'] == True]
    return gdp_data.index.values[0]
get_recession_start()

'2008q3'

In [25]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    gdp_data = import_data()
    recession_start = get_recession_start()
    
    gdp_data = gdp_data[gdp_data.index > recession_start]
    gdp_data['Recessionend'] = (gdp_data['GDP_Change'] > 0) & (gdp_data['GDP_Change'].shift(-1) > 0)
    return gdp_data[gdp_data['Recessionend'] == True].index[1]
get_recession_end()

'2009q4'

In [26]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    gdp_data = import_data()
    start = get_recession_start()
    end = get_recession_end()
    gdp_data = gdp_data[(gdp_data.index > start) & (gdp_data.index < end)]
    return np.argmin(gdp_data['GDP'])
    #return gdp_data
get_recession_bottom()

'2009q2'

In [27]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    """abb = [key for key in states]
    state = [states[key] for key in states]
    df = pd.DataFrame({'key': abb, 'State':state})
    
    housing = pd.merge( housing,df, how = "inner", left_on= 'State', right_on='key' )
    housing.rename(columns = {'State_y':'State'}, inplace = True)"""
    
    housing.replace({'State':states},inplace=True)
    housing = housing.set_index(['State', 'RegionName'])
    housing = housing.loc[: , '2000-01' : '2016-08']
    names = pd.Series(list(housing.columns))
    
    housing = housing.rename(columns=lambda x: pd.to_datetime(x))
    housing = housing.resample('Q',axis=1).mean()
    return housing.rename(columns=lambda x: str(x.to_period('Q')).lower())


In [31]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    towns = get_list_of_university_towns()
    house_price = convert_housing_data_to_quarters()
    rec_st = get_recession_start()
    rec_end = get_recession_end()
    rec_btm = get_recession_bottom()
    
    # adjust for duplicates
    house_price.drop_duplicates(keep=False,inplace=True)
    towns.drop_duplicates(keep=False,inplace=True)
    
    # calculate house price ratio
    house_price['Ratio'] = house_price['2008q2'].div(house_price['2009q2']) 
    
    # set index for uni towns to match house_price df
    towns = towns.set_index(['State', 'RegionName'])
    
    # find uni_towns
    uni_towns_ratio = house_price.loc[list(towns.index)]['Ratio']
    
    # find non-uni-towns
    ratio_diff = set(house_price.index) - set(uni_towns_ratio.index)
    non_uni_towns_ratio = house_price.loc[list(ratio_diff),:]['Ratio'] 
    
    t_stat, p_value = tuple(ttest_ind(uni_towns_ratio, non_uni_towns_ratio, nan_policy='omit'))

    result = t_stat < 0

    different = p_value < 0.01

    better = ["non-university town", "university town"]

    return (different, p_value, better[result])

run_ttest()

(True, 0.0027240637047531249, 'university town')